In [1]:
import akshare as ak

fund_etf_hist_em_df = ak.fund_etf_hist_em(symbol="518880", 
                                          period="daily", 
                                          start_date="20200101", end_date="20250512", adjust="")
print(fund_etf_hist_em_df)

  0%|          | 0/11 [00:00<?, ?it/s]

              日期     开盘     收盘     最高     最低       成交量           成交额    振幅  \
0     2020-01-02  3.371  3.370  3.377  3.368   3818438  1.287584e+09  0.27   
1     2020-01-03  3.392  3.422  3.428  3.390   3679150  1.254882e+09  1.13   
2     2020-01-06  3.496  3.497  3.510  3.493   4607805  1.613495e+09  0.50   
3     2020-01-07  3.472  3.466  3.475  3.447   3610570  1.249705e+09  0.80   
4     2020-01-08  3.547  3.531  3.560  3.511   5462982  1.926680e+09  1.41   
...          ...    ...    ...    ...    ...       ...           ...   ...   
1290  2025-05-06  7.596  7.612  7.659  7.570   9845270  7.499496e+09  1.19   
1291  2025-05-07  7.614  7.671  7.678  7.588   7518649  5.748539e+09  1.18   
1292  2025-05-08  7.679  7.557  7.732  7.538  11128511  8.503907e+09  2.53   
1293  2025-05-09  7.502  7.531  7.555  7.427   8873009  6.647216e+09  1.69   
1294  2025-05-12  7.341  7.376  7.399  7.341   6681992  4.930948e+09  0.77   

       涨跌幅    涨跌额    换手率  
0    -0.41 -0.014   4.77  
1     1.5

In [1]:
# -*- coding: utf-8 -*-
import vectorbt as vbt
import numpy as np
import pandas as pd
import numba
from loguru import logger
import sys
import os
import akshare as ak # <--- 新增akshare导入

# --- 全局配置 ---
strategy_name = "ETF_Momentum_Rotation_Akshare" # <--- 名称稍作修改以示区分
etf_symbols_original = ['518880.SH', '513100.SH', '513130.SH', '159915.SZ'] # 黄金, 纳指, 恒生科技, 创业板
# ETF代码给akshare使用 (不带后缀)
etf_codes_for_akshare = [s.split('.')[0] for s in etf_symbols_original]

start_date = '2020-01-01'
end_date = '2025-05-21'

data_directory = "Data_Akshare" # 如果需要本地缓存akshare数据 (本脚本直接在线获取)
output_dir = "results"

# --- 日志配置 ---
if not os.path.exists(output_dir):
    try:
        os.makedirs(output_dir)
        print(f"已创建输出目录: {output_dir}")
    except OSError as e:
        print(f"创建目录 {output_dir} 失败: {e}")
        sys.exit(1)

logger.remove()
log_format = "<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green> | <level>{level: <8}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - <level>{message}</level>"
logger.add(sys.stderr, level="INFO", format=log_format)
log_file_path = os.path.join(output_dir, f"{strategy_name}_backtest.log")
logger.add(log_file_path, level="DEBUG", rotation="10 MB", format=log_format)

logger.info(f"------ {strategy_name} 回测开始 ------")
logger.info(f"策略名称: {strategy_name}")
logger.info(f"原始ETF池: {etf_symbols_original}")

# --- 策略参数 ---
lookback_period = 25
annual_trading_days = 250
initial_cash = 100000
commission_rate = 0.0002
slippage_rate = 0.0005

# --- 辅助函数：动量计算 (Numba优化) ---
@numba.njit
def nb_calc_momentum_score(i, col, y_arr, annual_days): # 签名保持不变，接收vectorbt传入的参数
    n = len(y_arr)
    if np.isnan(y_arr).any() or n < 2: return np.nan
    
    log_y = np.log(y_arr)
    x = np.arange(n)
    
    # --- 开始修改：引入线性递增权重 ---
    weights = np.linspace(1.0, 2.0, n) # 权重从1线性增加到2
    
    # 加权线性回归计算 (参考Numpy的polyfit对于加权w的实现思路，或直接实现加权最小二乘法)
    # Numba环境下直接用np.polyfit(x, y, 1, w=weights)可能不支持或效率不高
    # 我们需要手动实现加权最小二乘法的斜率和截距计算
    
    w_sum = np.sum(weights)
    w_x_sum = np.sum(weights * x)
    w_y_sum = np.sum(weights * log_y)
    w_x2_sum = np.sum(weights * x**2)
    w_xy_sum = np.sum(weights * x * log_y)
    
    denominator = w_sum * w_x2_sum - w_x_sum**2
    if denominator == 0: return np.nan
    
    slope = (w_sum * w_xy_sum - w_x_sum * w_y_sum) / denominator
    intercept = (w_y_sum - slope * w_x_sum) / w_sum # 也可以是 (w_x2_sum * w_y_sum - w_x_sum * w_xy_sum) / denominator
    
    # 计算加权R²
    y_pred = slope * x + intercept
    weighted_residuals_sq = weights * (log_y - y_pred)**2
    
    # 加权均值
    weighted_mean_y = np.sum(weights * log_y) / w_sum
    weighted_ss_tot = np.sum(weights * (log_y - weighted_mean_y)**2)
    
    if weighted_ss_tot == 0: # 如果加权总平方和为0 (例如加权后的y值恒定)
        r_squared = 0.0 # 或1.0，取决于定义。如果预测完美，残差为0，R^2应为1。若y本身无波动，R^2通常无意义或为0。
                        # MarioC代码中是 np.sum(weights * (y - np.mean(y))**2)，这里用加权均值更一致。
                        # 如果 y 值本身恒定（即使加权后），weighted_ss_tot 会是0，导致除零。
                        # 如果 y 值恒定，log_y也恒定，那么slope会是0，y_pred也是恒定等于log_y。
                        # 此时 weighted_residuals_sq 会是0。如果 weighted_ss_tot 也是0，R^2可以定义为1（完美拟合常数）或0（无趋势）。
                        # 我们参考MarioC的实现，如果 y - np.mean(y) 部分加权后为0，则R^2可能出问题。
                        # 安全起见，如果 weighted_ss_tot 为0，且 weighted_residuals_sq 也为0，说明完美拟合常数，R^2=1。
                        # 如果 weighted_ss_tot 为0，但 weighted_residuals_sq 不为0（理论上不太可能），则R^2未定义或为0。
                        # 简单处理：若 weighted_ss_tot 为0，则 r_squared 为0 (表示没有可解释的方差，或趋势不明显)
        r_squared = 0.0
    else:
        r_squared = 1.0 - (np.sum(weighted_residuals_sq) / weighted_ss_tot)
        r_squared = max(0.0, r_squared) # 确保R²不为负

    # --- 修改结束 ---
        
    daily_factor = np.exp(slope)
    annualized_returns = daily_factor**annual_days - 1.0
    score = annualized_returns * r_squared
    
    if not np.isfinite(score): return np.nan
    return score

# --- 主执行程序 ---
if __name__ == "__main__":
    # --- 1. 加载数据 (使用 Akshare) ---
    logger.info(f"开始使用 akshare 加载数据，日期范围: {start_date} to {end_date}")
    
    # 转换日期格式为akshare需要的 'YYYYMMDD'
    ak_start_date = start_date.replace('-', '')
    ak_end_date = end_date.replace('-', '')

    all_etf_close_prices = {} # 存储每个ETF的收盘价Series，键为原始带后缀的symbol

    for i, ak_code in enumerate(etf_codes_for_akshare):
        original_symbol = etf_symbols_original[i] # 获取对应的原始带后缀的名称
        logger.debug(f"尝试从 akshare 加载 {original_symbol} (代码: {ak_code})...")
        try:
            etf_hist_df = ak.fund_etf_hist_em(
                symbol=ak_code,
                period="daily",
                start_date=ak_start_date,
                end_date=ak_end_date,
                adjust="hfq"  # 使用后复权数据
            )

            if etf_hist_df.empty:
                logger.warning(f"akshare 未返回 {original_symbol} (代码: {ak_code}) 的数据。")
                continue
            
            # 重命名列
            etf_hist_df.rename(columns={
                '日期': 'Date',
                '开盘': 'Open',
                '收盘': 'Close',
                '最高': 'High',
                '最低': 'Low',
                '成交量': 'Volume'
                # 其他列如 '成交额', '振幅', '涨跌幅', '涨跌额', '换手率' 可按需保留或重命名
            }, inplace=True)

            # 将'Date'列转换为datetime对象并设为索引
            etf_hist_df['Date'] = pd.to_datetime(etf_hist_df['Date'])
            etf_hist_df.set_index('Date', inplace=True)

            # 确保关键列是数值类型
            numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
            for col in numeric_cols:
                if col in etf_hist_df.columns:
                    etf_hist_df[col] = pd.to_numeric(etf_hist_df[col], errors='coerce')
                elif col == 'Volume': # 如果akshare没返回成交量，则补0
                     etf_hist_df[col] = 0
                else: # O,H,L,C是必须的
                    logger.error(f"ETF {original_symbol} 的akshare数据缺少关键列: {col}")
                    raise ValueError(f"ETF {original_symbol} 数据不完整")


            # 仅保留我们需要的 'Close' 列，并以原始带后缀的symbol命名
            if 'Close' in etf_hist_df.columns:
                all_etf_close_prices[original_symbol] = etf_hist_df['Close']
                logger.success(f"成功加载并处理了 {original_symbol} (代码: {ak_code}) 的数据。")
            else:
                logger.warning(f"ETF {original_symbol} 处理后缺少 'Close' 列。")

        except Exception as e_ak:
            logger.error(f"使用 akshare 加载 {original_symbol} (代码: {ak_code}) 时出错: {e_ak}")

    if not all_etf_close_prices:
        logger.error("未能从 akshare 加载任何有效的ETF数据。请检查ETF代码、日期范围和网络连接。")
        sys.exit(1)

    # 合并所有ETF的收盘价到一个DataFrame
    price_df = pd.DataFrame(all_etf_close_prices)
    
    # 数据清洗：填充周末等缺失的交易日数据，然后删除完全没有数据的行/列
    # 先基于第一个有效ETF的索引重新索引所有列，然后填充
    if not price_df.empty:
        base_index = price_df.index # 使用合并后的索引，它应该包含了所有交易日
        price_df = price_df.reindex(base_index).ffill() # 向前填充
        price_df = price_df.dropna(axis=0, how='all') # 删除所有列都为NaN的行（通常不会发生在此处）
        price_df = price_df.dropna(axis=1, how='all') # 删除数据加载后完全是NaN的ETF列
        # 进一步确保开始阶段没有NaN（动量计算对NaN敏感）
        # price_df = price_df.dropna(axis=0, how='any', subset=price_df.columns[:1]) # 基于第一个ETF确保起始数据完整

    if price_df.empty or price_df.shape[1] < 2: # 需要至少两个ETF才能轮动
        logger.error(f"数据加载或处理后，有效的ETF数量 ({price_df.shape[1]}) 少于2个，无法进行轮动。")
        sys.exit(1)
    
    # 更新实际参与轮动的ETF列表名称
    final_etf_symbols = price_df.columns.tolist()
    logger.info(f"最终使用的数据范围: {price_df.index.min()} 到 {price_df.index.max()}. 共 {len(price_df)} 行.")
    logger.info(f"最终参与轮动的ETF: {final_etf_symbols}")

    # --- 2. 计算动量得分 ---
    logger.info(f"开始计算动量得分，回顾期: {lookback_period} 天...")
    try:
        momentum_scores = price_df.vbt.rolling_apply(
            lookback_period,              # window (positional)
            nb_calc_momentum_score,       # apply_func_nb (positional)
            annual_trading_days           # *args (positional)
            # Removed args=, use_raw=, cache_func= based on documentation
        )
        logger.success("动量得分计算完成。")
    except Exception as e:
        logger.error(f"计算动量得分时出错: {e}")
        sys.exit(1)

    # --- 3. 生成交易信号
    logger.info("根据动量得分生成目标权重 (安全摸狗逻辑)...")
    try:
        # --- 开始修改：加入安全区间过滤 ---
        # 1. 复制一份动量得分用于操作，避免修改原始的 momentum_scores
        filtered_momentum_scores = momentum_scores.copy()

        # 2. 应用安全区间过滤条件: (0, 5]
        # 将不符合条件的得分设为负无穷小或NaN，这样它们在后续排名中会排在最后或被忽略
        # MarioC的逻辑是直接筛选DataFrame，我们这里通过修改得分来实现类似效果，以便后续排名
        condition = (filtered_momentum_scores > 0) & (filtered_momentum_scores <= 5)
        # 对于不满足条件的，我们给一个非常低的值，确保它们不会被选中
        # 或者，可以直接将它们设为NaN，rank函数会处理NaN
        filtered_momentum_scores[~condition] = np.nan 
        
        logger.info(f"应用安全区间 (0, 5] 过滤后的动量得分 (部分显示NaN为不合格):\n{filtered_momentum_scores.tail()}")

        # 3. 对过滤后的得分进行排名
        #   如果所有ETF都被过滤掉（即filtered_momentum_scores该行全是NaN），则ranks对应行也会是NaN
        ranks = filtered_momentum_scores.rank(axis=1, ascending=False, method='first')
        
        # 4. 生成目标权重：选择排名第一的（如果存在）
        #   如果某行ranks全是NaN，那么np.where(ranks == 1,...) 的结果在该行仍将是False(0.0)
        target_weights = pd.DataFrame(np.where(ranks == 1, 1.0, 0.0),
                                      index=price_df.index, 
                                      columns=price_df.columns)
        
        # 检查是否有任何一天选出了ETF (即是否有权重为1的情况)
        # 如果某天 target_weights.sum(axis=1) == 0，则表示当天无ETF可选，即空仓
        days_with_selection = target_weights.sum(axis=1) > 0
        logger.info(f"在 {days_with_selection.sum()} 天中选出了ETF进行持有 (共 {len(days_with_selection)} 个交易日)。")
        if days_with_selection.sum() == 0:
            logger.warning("警告：在整个回测期间，根据安全摸狗逻辑，没有选出任何ETF持有！请检查过滤条件或市场状况。")

        # --- 修改结束 ---
        logger.success("目标权重生成完成。")
    except Exception as e:
        logger.error(f"生成目标权重时出错: {e}")
        sys.exit(1)

    # 以黄金ETF (518880.SH) 为基准
    if '518880.SH' in price_df.columns:
        gold_etf_prices = price_df['518880.SH'].copy()
        # 确保基准价格序列与投资组合的索引对齐 (通常在price_df层面已经对齐)
        # gold_etf_prices = gold_etf_prices.reindex(portfolio.wrapper.index, method='ffill').fillna(method='bfill') # 对齐索引并填充
        benchmark_gold_returns = gold_etf_prices.vbt.to_returns()
        logger.info("已准备黄金ETF (518880.SH) 作为基准的收益率数据。")
    else:
        logger.warning("ETF池中未找到黄金ETF (518880.SH)，无法将其设为基准。")
        benchmark_gold_returns = None

    # 以纳指ETF (513100.SH) 为基准
    if '513100.SH' in price_df.columns:
        nasdaq_etf_prices = price_df['513100.SH'].copy()
        # nasdaq_etf_prices = nasdaq_etf_prices.reindex(portfolio.wrapper.index, method='ffill').fillna(method='bfill') # 对齐索引并填充
        benchmark_nasdaq_returns = nasdaq_etf_prices.vbt.to_returns()
        logger.info("已准备纳指ETF (513100.SH) 作为基准的收益率数据。")
    else:
        logger.warning("ETF池中未找到纳指ETF (513100.SH)，无法将其设为基准。")
        benchmark_nasdaq_returns = None


    vbt.settings['portfolio']['stats']['settings']['benchmark_rets'] = benchmark_nasdaq_returns
    
    # --- 4. 执行组合回测 ---
    logger.info(f"开始执行VectorBT组合回测，初始资金: {initial_cash:.2f}...")
    try:
        portfolio = vbt.Portfolio.from_orders(
            close=price_df, size=target_weights, size_type='targetpercent',
            group_by=True, cash_sharing=True, fees=commission_rate,
            slippage=slippage_rate, freq='D'
        )
        logger.success("组合回测执行完毕。")
    except Exception as e:
        logger.error(f"执行组合回测时出错: {e}")
        sys.exit(1)
    # 在 portfolio 回测完成之后，生成统计和绘图之前


2025-05-21 15:17:17.936 | INFO     | __main__:<module>:38 - ------ ETF_Momentum_Rotation_Akshare 回测开始 ------
2025-05-21 15:17:17.939 | INFO     | __main__:<module>:39 - 策略名称: ETF_Momentum_Rotation_Akshare
2025-05-21 15:17:17.940 | INFO     | __main__:<module>:40 - 原始ETF池: ['518880.SH', '513100.SH', '513130.SH', '159915.SZ']
2025-05-21 15:17:17.943 | INFO     | __main__:<module>:112 - 开始使用 akshare 加载数据，日期范围: 2020-01-01 to 2025-05-21


  0%|          | 0/11 [00:00<?, ?it/s]

2025-05-21 15:17:19.378 | SUCCESS  | __main__:<module>:166 - 成功加载并处理了 518880.SH (代码: 518880) 的数据。
2025-05-21 15:17:19.511 | SUCCESS  | __main__:<module>:166 - 成功加载并处理了 513100.SH (代码: 513100) 的数据。
2025-05-21 15:17:19.622 | SUCCESS  | __main__:<module>:166 - 成功加载并处理了 513130.SH (代码: 513130) 的数据。
2025-05-21 15:17:19.784 | SUCCESS  | __main__:<module>:166 - 成功加载并处理了 159915.SZ (代码: 159915) 的数据。
2025-05-21 15:17:19.801 | INFO     | __main__:<module>:196 - 最终使用的数据范围: 2020-01-02 00:00:00 到 2025-05-21 00:00:00. 共 1302 行.
2025-05-21 15:17:19.803 | INFO     | __main__:<module>:197 - 最终参与轮动的ETF: ['518880.SH', '513100.SH', '513130.SH', '159915.SZ']
2025-05-21 15:17:19.804 | INFO     | __main__:<module>:200 - 开始计算动量得分，回顾期: 25 天...
2025-05-21 15:17:23.532 | SUCCESS  | __main__:<module>:208 - 动量得分计算完成。
2025-05-21 15:17:23.534 | INFO     | __main__:<module>:214 - 根据动量得分生成目标权重 (安全摸狗逻辑)...
2025-05-21 15:17:23.573 | INFO     | __main__:<module>:228 - 应用安全区间 (0, 5] 过滤后的动量得分 (部分显示NaN为不合格):
            518880

In [2]:
etf_symbols_original

['518880.SH', '513100.SH', '513130.SH', '159915.SZ']

In [3]:
portfolio.stats()

Start                         2020-01-02 00:00:00
End                           2025-05-21 00:00:00
Period                         1302 days 00:00:00
Start Value                                 100.0
End Value                              338.406069
Total Return [%]                       238.406069
Benchmark Return [%]                    139.04321
Max Gross Exposure [%]                      100.0
Total Fees Paid                          6.927004
Max Drawdown [%]                        22.571891
Max Drawdown Duration           353 days 00:00:00
Total Trades                                  101
Total Closed Trades                           100
Total Open Trades                               1
Open Trade PnL                           6.977555
Win Rate [%]                                 50.0
Best Trade [%]                          23.714922
Worst Trade [%]                        -10.900036
Avg Winning Trade [%]                    5.263154
Avg Losing Trade [%]                    -2.540067


In [4]:
portfolio.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,159915.SZ,2020-02-13,50.266619,1.988994,0.019996,Buy
1,1,159915.SZ,2020-02-21,50.266619,2.143928,0.021554,Sell
2,2,159915.SZ,2020-02-24,49.435745,2.179089,0.021545,Buy
3,3,159915.SZ,2020-03-10,49.435745,2.061969,0.020387,Sell
4,4,518880.SH,2020-03-11,27.917561,3.649824,0.020379,Buy
...,...,...,...,...,...,...,...
196,196,513130.SH,2025-03-10,401.167813,0.793396,0.063657,Buy
197,197,513130.SH,2025-03-21,401.167813,0.760620,0.061027,Sell
198,198,518880.SH,2025-03-24,44.964838,6.783390,0.061003,Buy
199,199,518880.SH,2025-05-12,44.964838,7.372312,0.066299,Sell


In [7]:
portfolio.plot_trades(column='518880.SH')

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '9783c15f-b4d2-46e6-9633-adc7405c040e',
              'x': array([datetime.datetime(2020, 1, 2, 0, 0),
                          datetime.datetime(2020, 1, 3, 0, 0),
                          datetime.datetime(2020, 1, 6, 0, 0), ...,
                          datetime.datetime(2025, 5, 19, 0, 0),
                          datetime.datetime(2025, 5, 20, 0, 0),
                          datetime.datetime(2025, 5, 21, 0, 0)], shape=(1302,), dtype=object),
              'y': array([3.37 , 3.422, 3.497, ..., 7.222, 7.209, 7.439], shape=(1302,))},
             {'customdata': array([[0, 0, 27.91756085855497, 0.02037883672859664, 'Long'],
                                   [1, 1, 24.685671728618413, 0.018157980307805133, 'Long'],
                                   [2, 2, 25.214260460226708, 0.019349007441874733, 'Long'